# Addendum

It turns out there's a much simpler way of determining the interest rate by just calculating it from the available variables in the dataset!

First some standard imports

In [1]:
#pandas, numpy and math for managing and manipulating our data
import pandas as pd
import numpy as np
import math

#matplotlib for data visualisation
import matplotlib
import matplotlib.pyplot as plt

First I load the dataset and then drop any duplicates there might be. The customer ID is set to be the index or unique identifier of the dataset

In [2]:
dataframe = pd.read_csv('DataScienceAssignment/DataScienceAssignment.csv')
dataframe.drop_duplicates(inplace=True)
dataframe.set_index('id',inplace=True)

THe problem turns out to be incredibly straightforward because monthly payments can be calculated using the formula

$A = P\frac{r(1+r)^n}{(1+r)^n - 1}$

Where $A$ is the monthly payment, $P$ is the loan amount, $r$ is the interest per month (int_rate / 12) and n is the total number of payments ( 36 or 60) and so all the information is already in the dataframe.

In [3]:
# add a new columns with the number of months as a numeric
dataframe['nmths'] = np.where(dataframe.term == ' 36 months', 36, 60 )

#the total amount that will be paid over the coruse of the loan
dataframe['payment_total'] = dataframe.nmths * dataframe.installment

# drop any entries where the total payment is actually less than the funded amounts as this doesn't make sense
dataframe.drop(dataframe[dataframe.payment_total < dataframe.funded_amnt ].index, inplace=True)

# calculate the monthly amount (this is just for sanity checking)
dataframe['monthly_amount'] = dataframe.funded_amnt * ( (0.01 * dataframe.int_rate/12.0) * np.power( 1 + (0.01 * dataframe.int_rate/12.0), dataframe.nmths )) / ( np.power(1 + (0.01 * dataframe.int_rate/12.0), dataframe.nmths) -1 )

#there is enough information here now to just calculate the rate from the number of installments, the loan amounts, and the installment amounts
# however trying to solve for r in the above formula can only be done numerically, but luckily there is a function in numpy financial to do this!
import numpy_financial as npf
dataframe['interest_rate_calc'] = npf.rate(dataframe.nmths, -dataframe.installment, dataframe.funded_amnt, 0) * 12.0 * 100.0




Now I'll just make a scatter plot to show how nicely these line up. There are a handful of outliers, these look like mistakes in the data entry. I could remove them as I did above but it's really only superficial as we're not "training" our model.

In [1]:
fig,ax1 = plt.subplots()
ax1.plot([0, 35], [0, 35], linestyle='--', color='red')
ax1.scatter(dataframe['int_rate'], dataframe['interest_rate_calc'], color='blue', alpha=0.2)
ax1.set_xlabel('True Interest Rate')
ax1.set_ylabel('Calculated Interest Rate')

plt.show()

NameError: name 'plt' is not defined

Calculate a score for good to measure to compare!

In [5]:
X = dataframe['interest_rate_calc']
y = dataframe['int_rate']

print("root mean squared error  : %.3f" % math.sqrt((np.power(X - y,2)).mean()))

root mean squared error  : 0.042


Better than best results from before using 3 instead of O(2k) features, clearly thinking about the problem is the best solution!

Output new predictions for the test set.

In [6]:
dataframe_test = pd.read_csv('DataScienceAssignment/DataScienceAssignment_test.csv')
dataframe_test.drop_duplicates(inplace=True)
dataframe_test.set_index('id',inplace=True)
dataframe_test['nmths'] = np.where(dataframe_test.term == ' 36 months', 36, 60 )

test_predictions = npf.rate(dataframe_test.nmths, -dataframe_test.installment, dataframe_test.funded_amnt, 0) * 12.0 * 100.0

final_test_preds = pd.DataFrame.from_dict({'id' : dataframe_test.index, 'int_rate' : test_predictions})
final_test_preds.to_csv('DataScienceAssignment/DataScienceAssignment_test_predictions_new.csv')